In [3]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
import numpy.linalg as ln

In [4]:
tree = et.parse('/content/padgett.xml')
root = tree.getroot()

In [5]:
data = []

In [6]:
for x in root.findall('MetaNetwork'):
  for child in x.find('networks')[0]:
    data.append(child.attrib)

In [7]:
len(data)

256

In [8]:
df = pd.DataFrame(data)

In [9]:
df.head(20)

,source,target,type,value
0,ACCIAIUOL,ACCIAIUOL,double,0.0000
1,ACCIAIUOL,ALBIZZI,double,0.0000
2,ACCIAIUOL,BARBADORI,double,0.0000
3,ACCIAIUOL,BISCHERI,double,0.0000
4,ACCIAIUOL,CASTELLAN,double,0.0000
5,ACCIAIUOL,GINORI,double,0.0000
6,ACCIAIUOL,GUADAGNI,double,0.0000
7,ACCIAIUOL,LAMBERTES,double,0.0000
8,ACCIAIUOL,MEDICI,double,1.0000
9,ACCIAIUOL,PAZZI,double,0.0000


In [10]:
df.source.value_counts()

ACCIAIUOL    16
ALBIZZI      16
BARBADORI    16
BISCHERI     16
CASTELLAN    16
GINORI       16
GUADAGNI     16
LAMBERTES    16
MEDICI       16
PAZZI        16
PERUZZI      16
PUCCI        16
RIDOLFI      16
SALVIATI     16
STROZZI      16
TORNABUON    16
Name: source, dtype: int64

In [11]:
vec = np.array(df['value'], dtype = float)

In [12]:
L = vec.reshape(16,16)
print(L)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.]]


In [16]:
m = L.sum(axis = 1)
tmp = np.ones(16)
m_inv = np.diag(np.divide(tmp, m, out = np.zeros_like(tmp),  where = m != 0 ))
A = (L @ m_inv)
A = A.transpose()
p0 = np.ones(16)/16
eps = 10**-7
delta = 0.15
err = True
k = 0

while err == True:
  p1 = (1-delta)*(p0 @ A) + delta * np.ones(16)/16
  if ln.norm(p1-p0) < eps:
    err = False
  k += 1
  p0 = p1
  if k > 1000000 :
    err = False

print(p0)

[0.02874134 0.0741771  0.04715717 0.06455793 0.06499704 0.03039185
 0.09224798 0.02897769 0.13670365 0.03380051 0.0636331  0.009375
 0.06522587 0.05747176 0.08259229 0.0668247 ]


In [14]:
source_name = df.source.unique()
target_name = df.target.unique()

In [15]:
print (source_name)

['ACCIAIUOL' 'ALBIZZI' 'BARBADORI' 'BISCHERI' 'CASTELLAN' 'GINORI'
 'GUADAGNI' 'LAMBERTES' 'MEDICI' 'PAZZI' 'PERUZZI' 'PUCCI' 'RIDOLFI'
 'SALVIATI' 'STROZZI' 'TORNABUON']


In [53]:
res = pd.DataFrame(np.concatenate((source_name, p0), axis = 0).reshape(2,16)).T.rename(columns = {0:'source',1:'rank'}).sort_values(by = 'rank', ascending = False)

In [56]:
res.head(16)

,source,rank
8,MEDICI,0.136704
6,GUADAGNI,0.092248
14,STROZZI,0.082592
1,ALBIZZI,0.074177
15,TORNABUON,0.066825
12,RIDOLFI,0.065226
4,CASTELLAN,0.064997
3,BISCHERI,0.064558
10,PERUZZI,0.063633
13,SALVIATI,0.057472
